In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import itertools

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from numpy import ones,vstack
from numpy.linalg import lstsq
import matplotlib.pyplot as plt

import statsmodels.api as sm;

pd.options.display.max_columns = 150;

Ture = True;

In [2]:
typedict = {#'PID' : 'nominal',
            'SalePrice' : 'Continuous',
            #Matt
            'LotFrontage' : 'Continuous', 
            'LotArea' : 'Continuous',
            'maybe_LotShape' : 'Nominal',
            'LandSlope' : 'Nominal', 
            'LandContour' : 'Nominal', 
            'maybe_MSZoning' : 'Nominal', 
            'Street_paved' : 'Nominal', 
            'Alley' : 'Nominal',
            'Neighborhood' : 'Nominal', 
            'Foundation' : 'Nominal',
            'Utilities' : 'Nominal',
            'Heating' : 'Nominal',
            'HeatingQC_nom' : 'Ordinal',
            'CentralAir' : 'Nominal',
            'Electrical' : 'Nominal',
            'HeatingQC_ord' : 'Ordinal',
            'LotShape_com' : 'Nominal',
            'MSZoning_com' : 'Nominal',
            'LF_Near_NS_RR' : 'Nominal',
            'LF_Near_Positive_Feature' : 'Nominal',
            'LF_Adjacent_Arterial_St' : 'Nominal',
            'LF_Near_EW_RR' : 'Nominal',
            'LF_Adjacent_Feeder_St' : 'Nominal',
            'LF_Near_Postive_Feature' : 'Nominal',
            'Heating_com' : 'Nominal',
            'Electrical_com' : 'Nominal',
            'LotConfig_com' : 'Nominal', 
            'LotFrontage_log' : 'Continuous',
            'LotArea_log' : 'Continuous',
            #Oren 
            'MiscFeature': 'Nominal',
            'Fireplaces': 'Discrete',
            'FireplaceQu': 'Ordinal',
            'PoolQC': 'Ordinal',
            'PoolArea': 'Continuous',
            'PavedDrive': 'Nominal',
            'ExterQual': 'Ordinal',
            'OverallQual': 'Ordinal',
            'maybe_OverallCond': 'Ordinal',
            'MiscVal': 'Continuous',
            'YearBuilt': 'Discrete',
            'YearRemodAdd': 'Discrete',
            'KitchenQual': 'Ordinal',
            'Fence': 'Ordinal',
            'RoofStyle': 'Nominal',
            'RoofMatl': 'Nominal',
            'maybe_ExterCond': 'Ordinal',
            'maybe_MasVnrType': 'Nominal',
            'MasVnrArea': 'Continuous',
            #Mo
            'BsmtQual_ord': 'Ordinal',
            'BsmtCond_ord': 'Ordinal',
            'BsmtExposure_ord': 'Ordinal',
            'TotalBsmtSF': 'Continuous',
            'BSMT_GLQ':'Continuous', 
            'BSMT_Rec':'Continuous',
            'maybe_BsmtUnfSF': 'Continuous',
            'maybe_BSMT_ALQ':'Continuous',
            'maybe_BSMT_BLQ':'Continuous', 
            'maybe_BSMT_LwQ':'Continuous', 
            #Deck
            'WoodDeckSF':'Continuous', 
            'OpenPorchSF':'Continuous', 
            'ScreenPorch':'Continuous',
            'maybe_EnclosedPorch':'Continuous',
            'maybe_3SsnPorch':'Continuous',
            #Garage
            'GarageFinish':'Nominal', 
            'GarageYrBlt':'Continuous',
            'GarageCars':'Ordinal',
            'GarageArea':'Continuous',
            'GarageType_con':'Nominal',
            'maybe_GarageQual':'Nominal', 
            'maybe_GarageCond':'Nominal',

            # Hao-Wei
            "SaleType": "Nominal",
            "BldgType": "Nominal",
            "Functional_ord": "Ordinal", # Changed from "Functional"
            "1stFlrSF": "Continuous",
            "2ndFlrSF": "Continuous",
            "maybe_LowQualFinSF": "Continuous", # Rejectable p-value
            "GrLivArea": "Cbontinuous",
            "BsmtFullBath": "Discrete",
            "maybe_BsmtHalfBath": "Discrete", # Rejectable p-value
            "FullBath": "Discrete",
            "maybe_HalfBath": "Discrete",
            "BedroomAbvGr": "Discrete",
            "KitchenAbvGr": "Discrete",
            "TotRmsAbvGrd": "Discrete",
            "MoSold": "Discrete", # Rejectable p-value
            "YrSold": "Discrete", # Rejectable p-value
            "1stFlrSF_log": "Continuous",
            "2ndFlrSF_log": "Continuous",
            "GrLivArea_log": "Continuous",
            "number_floors": "Discrete",
            "attic": "Ordinal",
            "PUD": "Nominal",
            #### Whose?
            "SaleCondition": "Nominal",
            "SalePrice_log": "Continuous",
            "MS_coded": "Nominal",
            "sold_datetime": "Discrete",
    
            # Used locally in this notebook
            "Months_Elapsed": "Discrete"
}

In [3]:
attic_dict = {"No attic": 0, "Finished": 2, "Unfinished": 1};
fence_dict = {"No Fence": 0, "Minimum Privacy": 3, "Good Privacy": 4, "Good Wood": 2 , "Minimum Wood/Wire": 1};
PoolQC_dict = {0:0, "0":0, "Fa": 1, "TA":2, "Gd":3, "Ex":4};

In [4]:
housing = pd.read_csv('../data/ames_housing_price_data_v2.csv', index_col = 0);

In [5]:
y = housing["SalePrice"];
ylog = housing["SalePrice_log"];

x = housing.drop(["SalePrice", "SalePrice_log"], axis = 1);

In [9]:
np.sum(x.isnull(), axis = 0)[:50]

GrLivArea               0
maybe_MSZoning          0
LotFrontage           462
LotArea                 0
Street_paved            0
Alley                   0
maybe_LotShape          0
LandContour             0
Utilities               0
LandSlope               0
Neighborhood            0
BldgType                0
OverallQual             0
maybe_OverallCond       0
YearBuilt               0
YearRemodAdd            0
RoofStyle               0
RoofMatl                0
maybe_MasVnrType        0
MasVnrArea             14
ExterQual               0
maybe_ExterCond         0
Foundation              0
maybe_BsmtUnfSF         0
TotalBsmtSF             0
Heating                 0
HeatingQC_nom           0
CentralAir              0
Electrical              1
1stFlrSF                0
2ndFlrSF                0
maybe_LowQualFinSF      0
BsmtFullBath            0
maybe_BsmtHalfBath      0
FullBath                0
maybe_HalfBath          0
BedroomAbvGr            0
KitchenAbvGr            0
KitchenQual 